# Momentum_Strategy

In [ ]:
# 设定data路径
import os

os.chdir(r'F:\Study\Study Files\05大三上\金融统计分析\实验课1\1_Rawdata')

current_path = os.getcwd()
print(current_path)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_raw = pd.read_csv('TRD_Mnth.csv')
data_raw = data_raw.drop(['Markettype'], axis=1)

## 数据清洗

### 整体描述性统计

In [ ]:
data_raw.head()

In [ ]:
des = data_raw.describe(include='all')
print(des)

In [ ]:
# # 删除缺失值
# data_raw = data_raw.dropna()

# des = data_raw.describe(include='all')
# print(des)

问题：
- 每只股票第一个月的收益率为空值
- 每只股票对应的月份并非连续的，需要补充

### 检查`Stkcd`对应月份是否连续
- 结果：没有任何股票代码对应的月份是完全连续的

In [ ]:
# # 创建data_result的副本
# data_result_copy = data_result.copy()

# # 提取Trdmnt中的月份
# data_result_copy['month'] = data_result_copy['Trdmnt'].dt.month

# # 定义一个函数，检查月份是否连续
# def is_month_consecutive(group):
#     sorted_months = group.sort_values()
#     diff = np.diff(sorted_months)
#     return np.all(diff == 1) or np.all(diff == -11)

# # 按照Stkcd进行分组，在组内检查month是否连续
# consecutive_check = data_result_copy.groupby('Stkcd')['month'].apply(is_month_consecutive)

# print(consecutive_check)

# # 筛选出月份连续的股票代码
# consecutive_stkcd = consecutive_check[consecutive_check].index
# print(consecutive_stkcd)

# # 保留月份连续的股票代码的数据
# data_1 = data_result_copy[data_result_copy['Stkcd'].isin(consecutive_stkcd)]

# # data_1现在包含了符合条件的且月份连续的股票代码的数据
# data_1.head()

### 填充数据—补充缺失月份的数据
> 逻辑：收益率取值为0，市值与上月相同

In [ ]:
# import pandas as pd
# import numpy as np

# # 确保'Trdmnt'列是datetime类型
# data_raw['Trdmnt'] = pd.to_datetime(data_raw['Trdmnt'], format='%Y-%m')

# # 对数据进行排序
# data_raw = data_raw.sort_values(by=['Stkcd', 'Trdmnt'])

# # 使用groupby和apply函数实现数据的填充
# def fill_data(df):
#     # 生成一个完整的月份范围
#     all_months = pd.date_range(df['Trdmnt'].min(), df['Trdmnt'].max(), freq='M')
    
#     # 建立一个新的DataFrame来储存完整的月份和原始数据的匹配
#     new_df = pd.DataFrame(all_months, columns=['Trdmnt'])
#     new_df['Stkcd'] = df.iloc[0]['Stkcd']
    
#     # 将新数据和旧数据合并，并填充缺失值
#     merged_df = pd.merge(new_df, df, on=['Trdmnt', 'Stkcd'], how='left')
#     merged_df['Msmvosd'].fillna(method='ffill', inplace=True) # 填充市值
#     merged_df['Mretwd'].fillna(0, inplace=True)               # 填充回报率
    
#     # 确保第一个月的回报率为0
#     merged_df.loc[merged_df['Trdmnt'] == merged_df['Trdmnt'].min(), 'Mretwd'] = 0
    
#     return merged_df

# # 应用函数并连接结果
# filled_data = pd.concat([fill_data(df) for _, df in data_raw.groupby('Stkcd')], ignore_index=True)

# # 保存处理后的数据
# filled_data.to_csv("filled_TRD_Mnth.csv", index=False)

## 策略构建

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 读取数据
data = pd.read_csv('TRD_Mnth.csv')

# 确保数据按股票代码和日期排序
data = data.sort_values(['Stkcd', 'Trdmnt'])
data['Trdmnt'] = pd.to_datetime(data['Trdmnt'])

# 设置回测期
data = data[(data['Trdmnt'] >= '1992-01-01') & (data['Trdmnt'] <= '2021-12-31')]

# 初始化一个3x3的DataFrame来保存不同策略的收益
result = pd.DataFrame(index=[1,3,6], columns=[1,3,6])

# 开始循环：不同形成期和持有期
for form_period in [1,3,6]:
    for hold_period in [1,3,6]:
        
        all_returns = []
        
        # 遍历每一个可以进行交易的日期
        for end_date in pd.date_range(start='1992-01-01', end='2021-12-31', freq='M'):
            start_date = end_date - pd.DateOffset(months=form_period)  # 形成期起始日期
            hold_end_date = end_date + pd.DateOffset(months=hold_period)  # 持有期结束日期
            
            if hold_end_date > data['Trdmnt'].max():
                continue
            
            # 计算形成期内的累计收益
            mask = (data['Trdmnt'] >= start_date) & (data['Trdmnt'] < end_date)
            form_returns = data.loc[mask].groupby('Stkcd')['Mretwd'].apply(lambda x: (1 + x).prod() - 1)

            # 检查在持有期间是否有数据
            mask = (data['Trdmnt'] > end_date) & (data['Trdmnt'] <= hold_end_date)
            available_stocks = data.loc[mask]['Stkcd'].unique()
            
            # 仅保留那些在持有期内有数据的股票
            form_returns = form_returns[form_returns.index.isin(available_stocks)]
            
            # 从剩下的股票中确定赢家和输家
            
            winners = form_returns.nlargest(10).index
            losers = form_returns.nsmallest(10).index
            
            # 计算持有期收益
            mask = (data['Trdmnt'] > end_date) & (data['Trdmnt'] <= hold_end_date)
            hold_returns = data.loc[mask].groupby(['Stkcd', 'Trdmnt'])['Mretwd'].apply(lambda x: ((1 + x).prod())**(1/hold_period) - 1).unstack().mean(axis=1)
            
            winner_returns = hold_returns.loc[winners].mean()
            loser_returns = hold_returns.loc[losers].mean()
            
            # 计算策略收益：赢家收益 - 输家收益
            strategy_return = winner_returns - loser_returns
            
            all_returns.append(strategy_return)
            
        cumulative_returns = all_returns + 1  # 每个月的累计收益率
        geometric_mean = np.prod(cumulative_returns) - 1  # 几何平均收益率
        annual_returns = (geometric_mean + 1) ** 12 - 1  # 年收益率

        result.loc[form_period, hold_period] = annual_returns


## 完整代码

In [ ]:
# 设定data路径
import os

os.chdir(r'F:\Study\Study Files\05大三上\金融统计分析\实验课1\1_Rawdata')

current_path = os.getcwd()
print(current_path)

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# 读取数据
data = pd.read_csv('TRD_Mnth.csv')
data = data.drop(['Markettype'], axis=1)

data.head()


# 确保数据按股票代码和日期排序
data = data.sort_values(['Stkcd', 'Trdmnt'])
data['Trdmnt'] = pd.to_datetime(data['Trdmnt'])

# 设置回测期
data_trade = data[(data['Trdmnt'] >= '2021-01-01') & (data['Trdmnt'] <= '2022-12-31')]

# 初始化一个3x3的DataFrame来保存不同策略的收益
result = pd.DataFrame(index=[1,3,6], columns=[1,3,6])

# 获取唯一的年月，作为回测的月份点
unique_year_month = data_trade['Trdmnt'].dt.to_period('M').unique()


# 开始循环：不同形成期和持有期
for form_period in [1,3,6]:
    for hold_period in [1,3,6]:
        
        all_returns = []
        
        # 遍历每一个唯一的年月
        for current_ym in unique_year_month:
            
            # 将年月转换为实际的日期
            end_date = pd.to_datetime(current_ym.to_timestamp())
            start_date = end_date - pd.DateOffset(months=form_period)  # 形成期起始日期
            hold_end_date = end_date + pd.DateOffset(months=hold_period)  # 持有期结束日期
            
            if hold_end_date > data['Trdmnt'].max():
                continue
            
            # 计算形成期内的累计收益
            mask = (data['Trdmnt'] > start_date) & (data['Trdmnt'] <= end_date)
            form_returns = data.loc[mask].groupby('Stkcd')['Mretwd'].apply(lambda x: (1 + x).prod() - 1)

            # 检查在持有期间是否有数据
            mask = (data['Trdmnt'] > end_date) & (data['Trdmnt'] <= hold_end_date)
            available_stocks = data.loc[mask]['Stkcd'].unique()
            
            # 仅保留那些在持有期内有数据的股票
            form_returns = form_returns[form_returns.index.isin(available_stocks)]
            
            # 从剩下的股票中确定赢家和输家
            
            winners = form_returns.nlargest(10).index
            losers = form_returns.nsmallest(10).index
            
            # 计算持有期收益
            mask = (data['Trdmnt'] > end_date) & (data['Trdmnt'] <= hold_end_date)
            hold_returns = data.loc[mask].groupby(['Stkcd', 'Trdmnt'])['Mretwd'].apply(lambda x: ((1 + x).prod())**(1/hold_period) - 1).unstack().mean(axis=1)
            
            winner_returns = hold_returns.loc[winners].mean()
            loser_returns = hold_returns.loc[losers].mean()
            
            # 计算策略收益：赢家收益 - 输家收益
            strategy_return = winner_returns - loser_returns
            
            all_returns.append(strategy_return)
            
        # 删除缺失值
        all_returns_clean = [x for x in all_returns if not np.isnan(x)]

        # 计算累计收益率、几何平均收益率和年收益率
        cumulative_returns = [x + 1 for x in all_returns_clean]
        geometric_mean = np.prod(cumulative_returns) - 1
        annual_returns = (geometric_mean + 1) ** 12 - 1

        result.loc[form_period, hold_period] = annual_returns

sns.heatmap(result.astype(float), annot=True, fmt=".2%", cmap="vlag", center=0)
plt.title("Momentum Strategy Returns (Jegadeesh and Titman, 1993)")
plt.xlabel("Holding Period (months)")
plt.ylabel("Formation Period (months)")
#plt.savefig("v2-2012.png")  # 将图像保存为heatmap.png
plt.show()